In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [3]:
df=pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

## Basic preprocessing

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df.shape

(50000, 2)

In [6]:
df['review'].str.lower()

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. <br /><br />the...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
49995    i thought this movie did a down right good job...
49996    bad plot, bad dialogue, bad acting, idiotic di...
49997    i am a catholic taught in parochial elementary...
49998    i'm going to have to disagree with the previou...
49999    no one expects the star trek movies to be high...
Name: review, Length: 50000, dtype: object

## Remove Html tags

In [7]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub('', text)


In [8]:
text = "<p><strong>Hello!</strong> This is a one-line text with <em>HTML</em> tags.</p>"


In [9]:
remove_html_tags(text)

'Hello! This is a one-line text with HTML tags.'

In [10]:
df['review'].apply(remove_html_tags)

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. The filming tec...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
49995    I thought this movie did a down right good job...
49996    Bad plot, bad dialogue, bad acting, idiotic di...
49997    I am a Catholic taught in parochial elementary...
49998    I'm going to have to disagree with the previou...
49999    No one expects the Star Trek movies to be high...
Name: review, Length: 50000, dtype: object

## Remove Urls

In [11]:
text1 = "Text1: Visit Google at https://www.google.com"
text2 = "Text2: Learn web development at https://www.w3schools.com"



In [12]:
def remove_url(text):
    pattern=re.compile(r'https?://\S+|www\.\S+')
    return pattern.sub(r'',text)

In [13]:
remove_url(text1)


'Text1: Visit Google at '

In [14]:
remove_url(text2)


'Text2: Learn web development at '

In [15]:
## remove punctuation

In [16]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
exclude=string.punctuation

In [18]:
def remove_punc(text):
    for char in exclude:
        text=text.replace(char,'')
    return text

In [19]:
text='string. with. punctuations?'

In [20]:
start=time.time()
print(remove_punc(text))
time1=time.time() - start 
print(time1*50000)
## but its too slow


string with punctuations
8.26120376586914


In [21]:
def remove_punc1(text):
    return text.translate(str.maketrans('','',exclude))

In [22]:
start=time.time()
remove_punc1(text)
time2=time.time() - start
print(time2*50000)

4.00543212890625


## Chat word treatment

In [23]:
chat_words = {
    "GN": "Good Night",
    "GM": "Good Morning",
    "LOL": "Laughing Out Loud",
    "BRB": "Be Right Back",
    "GTG": "Got To Go",
    "IDK": "I Don't Know",
    "BTW": "By The Way",
    "FYI": "For Your Information",
    "TMI": "Too Much Information",
    "OMG": "Oh My God",
    "LMAO": "Laughing My Ass Off",
    "ROFL": "Rolling On the Floor Laughing",
    "BFF": "Best Friends Forever",
    "YOLO": "You Only Live Once",
    "SMH": "Shaking My Head",
    "FOMO": "Fear Of Missing Out",
    "TL;DR": "Too Long; Didn't Read",
    "IMHO": "In My Humble Opinion",
    "AKA": "Also Known As",
    "WTF": "What The F***",
    "DM": "Direct Message",
    "CC": "Carbon Copy",
    "RSVP": "Répondez S'il Vous Plaît (Please Respond)",
    "CEO": "Chief Executive Officer",
    "FAQ": "Frequently Asked Questions",
    "VIP": "Very Important Person",
    "DIY": "Do It Yourself",
    "BR": "Best Regards",
    "ASAP": "As Soon As Possible",
    "NSFW": "Not Safe For Work",
    "TGIF": "Thank God It's Friday",
    "TBT": "Throwback Thursday",
    "LMK": "Let Me Know",
    "PM": "Private Message",
    "WIP": "Work In Progress",
    "LOLZ": "Laugh Out Loud (plural)",
    "IMO": "In My Opinion",
    "PPL": "People",
    "B4": "Before",
    "GR8": "Great",
    "THX": "Thanks",
    "PLZ": "Please",
    "CYA": "See You Around",
    "H8": "Hate",
    "L8R": "Later",
    "K": "Okay",
    "NVM": "Never Mind",
    "NP": "No Problem",
    "T2UL": "Talk To You Later",
    "2F2F": "Too Fast Too Furious",
    "XOXO": "Hugs and Kisses",
    "OMW": "On My Way",
    "WBU": "What About You",
    "4U": "For You",
    "2NITE": "Tonight"
}


In [24]:
def chat_conversation(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [25]:
chat_conversation('IMHO he is the best')

'In My Humble Opinion he is the best'

## Spelling Correction

In [26]:
text="please read the note book, and also like the notebook"

In [27]:
from textblob import TextBlob

In [28]:
incorrect_text='ceertain conditions during several ggenerations in the saame maner'
textBlb=TextBlob(incorrect_text)
textBlb.correct().string

'certain conditions during several generations in the same manner'

## Removing Stop Words

In [29]:
from nltk.corpus import stopwords

In [30]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [31]:
def remove_stopwords(text):
    new_text=[]
    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x=new_text[:]
    new_text.clear()
    return " ".join(x)


In [32]:
remove_stopwords("The cat is sitting on the mat in the corner of the room."
 )

'The cat  sitting   mat   corner   room.'

## Removing Emojis 
-- Remove
--Replace

In [33]:
import re

def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)


In [34]:
remove_emoji("loved the movie.it was 😑😑")

'loved the movie.it was '

In [35]:
import emoji
print(emoji.demojize('python is♥️'))

python is:heart_suit:


## Tokenization
-- depends on word level
-- depends also on sentence level

In [36]:
##  remember prefix,suffix,infix,exception

### Techniques for tokenization

#### using the split function

In [37]:
sent1='I am going to delhi'
sent1.split()

['I', 'am', 'going', 'to', 'delhi']

In [38]:
sent2='I am going to delhi. I will stay there for 3 days.let\'s hope the trip to be great'
sent2.split('.')

['I am going to delhi',
 ' I will stay there for 3 days',
 "let's hope the trip to be great"]

In [39]:
# problem with split function
sent3='I am going to delhi!'
sent3.split()

['I', 'am', 'going', 'to', 'delhi!']

In [40]:
sent4='where do think i should go? i have 3 day holiday'
sent4.split('.')  ## will not split

['where do think i should go? i have 3 day holiday']

### Regular Expression

In [41]:
import re 
sent3='I am going to delhi!'
tokens=re.findall("[\w']+",sent3)
tokens

['I', 'am', 'going', 'to', 'delhi']

In [42]:
text="""I woke up to the sound of birds chirping outside my window. After stretching a bit, I made some coffee and watched the sunrise light up the sky. Feeling refreshed, I started my day with a smile and a sense of calm."""
sentences=re.compile('[.!?]').split(text)
sentences

['I woke up to the sound of birds chirping outside my window',
 ' After stretching a bit, I made some coffee and watched the sunrise light up the sky',
 ' Feeling refreshed, I started my day with a smile and a sense of calm',
 '']

In [43]:
## NLTK

In [44]:
from nltk.tokenize import word_tokenize,sent_tokenize

In [53]:
sen1="I am going to visit delhi!"
sent2="For any inquiries, feel free to contact us at info@example.com."
sent3="i did my Ph.D in AI"
word_tokenize(sen1)

['I', 'am', 'going', 'to', 'visit', 'delhi', '!']

In [54]:
word_tokenize(sent2) ## it will split evrything which is not right sometime like for mail

['For',
 'any',
 'inquiries',
 ',',
 'feel',
 'free',
 'to',
 'contact',
 'us',
 'at',
 'info',
 '@',
 'example.com',
 '.']

In [55]:
## Spacy

In [56]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [57]:
doc1=nlp(sen1)
doc2=nlp(sent2)

In [58]:
for token in doc2:
    print(token)    ## perform weel on sent2 but not on set3

For
any
inquiries
,
feel
free
to
contact
us
at
info@example.com
.


## STeaming
-- Inflection= in grammer inflection is a modification of word to express different grammatical categories such as tense case,voice,aspect,person,number,gender and mood
-- Steaming= is the process of reducing inflection in words to their root forms such as mapping a group of words to the same stem even if the stem itself is not a valid word in language
(##word ko uske root form me lana)
-- STammer = algo used for steaming( we will user porter stammer but there are others also )
                      

In [59]:
from nltk.stem.porter import PorterStemmer

In [60]:
ps=PorterStemmer()
def stem_words(text):
    return " ".join([ps.stem(word) for word in text.split()])

In [62]:
sample="walk walks walking walked"
stem_words(sample)

'walk walk walk walk'

In [64]:
text="I woke up to the sound of birds chirping outside my window. After stretching a bit, I made some coffee and watched the sunrise light up the sky. Feeling refreshed, I started my day with a smile and a sense of calm"
print(text)
stem_words(text)  ## outside as outsid and so many words are there which is not good for dispaly to user for this we use lemmetization  because stemp word is not a english word

I woke up to the sound of birds chirping outside my window. After stretching a bit, I made some coffee and watched the sunrise light up the sky. Feeling refreshed, I started my day with a smile and a sense of calm


'i woke up to the sound of bird chirp outsid my window. after stretch a bit, i made some coffe and watch the sunris light up the sky. feel refreshed, i start my day with a smile and a sens of calm'

## Lemmetization (english word but it is slow)
__ root word belong to language

In [67]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer=WordNetLemmatizer()
sentence="I woke up to the sound of birds chirping outside my window. After stretching a bit, I made some coffee and watched the sunrise light up the sky. Feeling refreshed, I started my day with a smile and a sense of calm"
punctuations="?:!.,;"
sentence_words=nltk.word_tokenize(sentence)
for word in sentence_words:
    if word in punctuations:
        sentence_words.remove(word)
sentence_words
print("{0:20}{1:20}".format("Word","Lemma"))
for word in sentence_words:
    print("{0:20}{1:20}".format(word,wordnet_lemmatizer.lemmatize(word)))

Word                Lemma               
I                   I                   
woke                woke                
up                  up                  
to                  to                  
the                 the                 
sound               sound               
of                  of                  
birds               bird                
chirping            chirping            
outside             outside             
my                  my                  
window              window              
After               After               
stretching          stretching          
a                   a                   
bit                 bit                 
I                   I                   
made                made                
some                some                
coffee              coffee              
and                 and                 
watched             watched             
the                 the                 
sunrise         